In [8]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.decomposition import PCA
from sklearn.ensemble import (
    RandomForestRegressor, 
    RandomForestClassifier,  # Added this import
    GradientBoostingRegressor,
    GradientBoostingClassifier
)
from sklearn.linear_model import (
    LinearRegression, 
    Ridge, 
    Lasso, 
    ElasticNet, 
    LogisticRegression,
    SGDRegressor,
    SGDClassifier
)
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.svm import SVR, SVC
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (
    mean_squared_error, 
    r2_score, 
    mean_absolute_error,
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score
)
import json
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)



In [9]:
class MLPipeline:
    def __init__(self, json_path, data_path):
        """Initialize pipeline with paths to configuration and data files"""
        self.load_config(json_path)
        self.load_data(data_path)
        
    def load_config(self, json_path):
        """Load and parse the JSON configuration file"""
        try:
            with open(json_path, 'r') as f:
                self.config = json.load(f)
            self.design_state = self.config['design_state_data']
            logger.info("Successfully loaded configuration file")
        except Exception as e:
            logger.error(f"Error loading configuration file: {str(e)}")
            raise
            
    def load_data(self, data_path):
        """Load the dataset and prepare features and target"""
        try:
            self.df = pd.read_csv(data_path)
            self.target_col = self.design_state['target']['target']
            self.prediction_type = self.design_state['target']['prediction_type']
            
            self.X = self.df.drop(columns=[self.target_col])
            self.y = self.df[self.target_col]
            
            # Handle categorical target for classification
            if self.prediction_type == 'Classification':
                self.le = LabelEncoder()
                self.y = self.le.fit_transform(self.y)
            
            logger.info(f"Data loaded successfully. Shape: {self.df.shape}")
            return self.X, self.y
        except Exception as e:
            logger.error(f"Error loading data: {str(e)}")
            raise

    def create_feature_handlers(self):
        """Create feature handling transformers based on configuration"""
        transformers = []
        feature_config = self.design_state['feature_handling']
        
        for feature_name, config in feature_config.items():
            if not config['is_selected'] or feature_name == self.target_col:
                continue
                
            if config['feature_variable_type'] == 'numerical':
                # Create imputer
                if config['feature_details']['impute_with'] == 'Average of values':
                    imputer = SimpleImputer(strategy='mean')
                else:
                    imputer = SimpleImputer(
                        strategy='constant',
                        fill_value=config['feature_details']['impute_value']
                    )
                
                # Create pipeline for the feature
                feature_pipeline = Pipeline([
                    ('imputer', imputer)
                ])
                
                if config['feature_details']['rescaling'] != 'No rescaling':
                    feature_pipeline.steps.append(('scaler', StandardScaler()))
                
                transformers.append(
                    (f'transform_{feature_name}',
                     feature_pipeline,
                     [feature_name])
                )
            
            elif config['feature_variable_type'] == 'text':
                # Handle text features based on configuration
                # Add text handling logic here if needed
                pass
                
        return ColumnTransformer(transformers) if transformers else None

    def create_feature_reducer(self):
        """Create feature reduction transformer based on configuration"""
        reduction_config = self.design_state['feature_reduction']
        method = reduction_config['feature_reduction_method']
        n_features = int(reduction_config['num_of_features_to_keep'])
        
        reduction_methods = {
            'Tree-based': SelectKBest(score_func=mutual_info_regression, k=n_features),
            'Corr with Target': SelectKBest(score_func=f_regression, k=n_features),
            'PCA': PCA(n_components=n_features)
        }
        
        return reduction_methods.get(method)

    def get_model_configs(self):
        """Get selected models and their hyperparameter grids"""
        models = []
        algorithms = self.design_state['algorithms']
        
        # Define model mappings based on prediction type
        regression_models = {
            'RandomForestRegressor': (RandomForestRegressor, {
                'n_estimators': [10, 20, 30],
                'max_depth': [20, 25, 30],
                'min_samples_leaf': [5, 10]
            }),
            'LinearRegression': (LinearRegression, {
                'fit_intercept': [True, False]
            }),
            'RidgeRegression': (Ridge, {
                'alpha': [0.1, 0.5, 1.0],
                'max_iter': [500, 1000]
            }),
            'LassoRegression': (Lasso, {
                'alpha': [0.1, 0.5, 1.0],
                'max_iter': [500, 1000]
            }),
            'ElasticNetRegression': (ElasticNet, {
                'alpha': [0.1, 0.5, 1.0],
                'l1_ratio': [0.2, 0.5, 0.8]
            }),
            'DecisionTreeRegressor': (DecisionTreeRegressor, {
                'max_depth': [4, 5, 6, 7],
                'min_samples_leaf': [6, 12]
            })
        }
        
        classification_models = {
            'RandomForestClassifier': (RandomForestClassifier, {
                'n_estimators': [10, 20, 30],
                'max_depth': [20, 25, 30],
                'min_samples_leaf': [5, 10]
            }),
            'LogisticRegression': (LogisticRegression, {
                'C': [0.1, 1.0, 10.0],
                'max_iter': [1000]
            }),
            'DecisionTreeClassifier': (DecisionTreeClassifier, {
                'max_depth': [4, 5, 6, 7],
                'min_samples_leaf': [6, 12]
            })
        }
        
        model_mapping = regression_models if self.prediction_type == 'Regression' else classification_models
        
        for model_name, config in algorithms.items():
            if config['is_selected'] and model_name in model_mapping:
                model_class, param_grid = model_mapping[model_name]
                models.append((model_name, model_class(), param_grid))
        
        return models

    def evaluate_model(self, y_true, y_pred, model_name):
        """Evaluate model performance and log metrics"""
        if self.prediction_type == 'Regression':
            metrics = {
                'R² Score': r2_score(y_true, y_pred),
                'MSE': mean_squared_error(y_true, y_pred),
                'MAE': mean_absolute_error(y_true, y_pred),
                'RMSE': np.sqrt(mean_squared_error(y_true, y_pred))
            }
        else:
            metrics = {
                'Accuracy': accuracy_score(y_true, y_pred),
                'Precision': precision_score(y_true, y_pred, average='weighted'),
                'Recall': recall_score(y_true, y_pred, average='weighted'),
                'F1 Score': f1_score(y_true, y_pred, average='weighted')
            }
        
        logger.info(f"\nMetrics for {model_name}:")
        for metric_name, value in metrics.items():
            logger.info(f"{metric_name}: {value:.4f}")

    def run_pipeline(self):
        """Execute the complete machine learning pipeline"""
        try:
            # Split data
            X_train, X_test, y_train, y_test = train_test_split(
                self.X, self.y, test_size=0.2, random_state=42
            )
            
            # Create pipeline components
            feature_handler = self.create_feature_handlers()
            feature_reducer = self.create_feature_reducer()
            
            # Get model configurations
            models = self.get_model_configs()
            
            for model_name, model, param_grid in models:
                logger.info(f"\nTraining {model_name}...")
                
                # Build pipeline
                pipeline_steps = []
                if feature_handler:
                    pipeline_steps.append(('feature_handling', feature_handler))
                if feature_reducer:
                    pipeline_steps.append(('feature_reduction', feature_reducer))
                pipeline_steps.append(('model', model))
                
                pipeline = Pipeline(pipeline_steps)
                
                # Configure GridSearchCV
                grid_search = GridSearchCV(
                    pipeline,
                    {f'model__{k}': v for k, v in param_grid.items()},
                    cv=5,
                    scoring='neg_mean_squared_error' if self.prediction_type == 'Regression' else 'accuracy',
                    n_jobs=-1
                )
                
                # Fit and predict
                grid_search.fit(X_train, y_train)
                y_pred = grid_search.predict(X_test)
                
                # Log results
                logger.info(f"Best parameters: {grid_search.best_params_}")
                self.evaluate_model(y_test, y_pred, model_name)
                
        except Exception as e:
            logger.error(f"Error in pipeline execution: {str(e)}")
            raise

def main():
    # Initialize and run pipeline
    try:
        pipeline = MLPipeline("algoparams_from_ui.json", "iris.csv")
        pipeline.run_pipeline()
    except Exception as e:
        logger.error(f"Pipeline execution failed: {str(e)}")

if __name__ == "__main__":
    main()

INFO:__main__:Successfully loaded configuration file
INFO:__main__:Data loaded successfully. Shape: (150, 5)
INFO:__main__:
Training RandomForestRegressor...
C:\Users\Tanaya\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
90 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Tanaya\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Tanaya\Anaconda\lib\site-packages\sklearn\pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "C:\Users\